In [ ]:
!pip install git+https://github.com/PyTorchLightning/pytorch-lightning
import pytorch_lightning as pl
import torch
import matplotlib.pyplot as plt
import torch.nn.functional as F
import numpy as np
from torch.utils.data import DataLoader

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/PyTorchLightning/pytorch-lightning to /tmp/pip-req-build-vo9hp33n
  Running command git clone -q https://github.com/PyTorchLightning/pytorch-lightning /tmp/pip-req-build-vo9hp33n
  Running command git submodule update --init --recursive -q
  From https://github.com/PyTorchLightning/lightning-tutorials
   * branch            290fb466de1fcc2ac6025f74b56906592911e856 -> FETCH_HEAD
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 418 kB 5.3 MB/s 
     |████████████████████████████████| 140 kB 60.2 MB/s 
     |████████████████████████████████| 596 kB 50.0 MB/s 
     |████████████████████████████████| 1.1 MB 50.3 MB/s 
     |████████████████████████████████| 94 kB 4.2 MB/s 
     |████████████████████████████████| 144 kB 72.0 MB/s 
     |██████████████

In [ ]:
from skimage import io
class Dataset(torch.utils.data.Dataset):
  def __init__(self, X, y, trans, device):
    self.X = X 
    self.y = y 
    self.trans = trans
    self.device = device

  def __len__(self):
    return len(self.X)
  def __getitem__(self, ix):
    # cargar la imágen
    img = io.imread(self.X[ix], format = "jpg")
    # aplicar transformaciones
    if self.trans:
      #image_raw = tf.io.read_file(image_path)
#image = tf.image.decode_image(image_raw)
      img = self.trans(image=img)["image"]
    return torch.tensor(img /255.).float().view(3, 28, 28), torch.tensor(self.y[ix]).long()
    #return torch.tensor(self.X[ix]).float(), torch.tensor(self.y[ix]).long()

In [ ]:
path = '/content/drive/MyDrive/IA2/Dataset_basura'

In [ ]:
import os
clases = os.listdir(path);
imgs, labels = [], []

for i, lab in enumerate(clases):
  paths = os.listdir(f'{path}/{lab}')
  print(f'Categoría: {lab}. Imágenes: {len(paths)}')
  paths = [p for p in paths if p[-3:] == "jpg"]
  imgs += [f'{path}/{lab}/{img}' for img in paths]
  labels += [i]*len(paths)

Categoría: metales. Imágenes: 208
Categoría: cartones. Imágenes: 250
Categoría: botellas_de_plastico. Imágenes: 1
Categoría: Plastoformos. Imágenes: 265
Categoría: papel. Imágenes: 674
Categoría: BolsasPlasticas. Imágenes: 72


In [ ]:
from sklearn.model_selection import train_test_split
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
import albumentations as A
trans = A.Compose([
    A.Resize(28, 28)
])

class BasuraDataModule(pl.LightningDataModule):

    def __init__(self, batch_size: int = 64, Dataset = Dataset):
        super().__init__()
        self.batch_size = batch_size
        self.Dataset = Dataset

    def setup(self, stage=None):
        #X_tr, X_t, y_tr, y_t = train_test_split(imgs, labels, test_size=0.2, stratify=labels)
        #X_train, X_test, y_train, y_test = train_test_split(imgs, labels, test_size=0.1, stratify=labels)
        #mnist = fetch_openml('mnist_784')
        #print(mnist["data"])
        #print(mnist["target"])
        #X, y = mnist["data"], mnist["target"]
        X_train, X_test, y_train, y_test = imgs[:20000] , imgs[20000:], labels[:20000], labels[20000:]
        #X_train = X_tr
        #X_test = X_t
        #y_train = y_tr
        #y_test = y_t

        print(X_train)
        print(y_train)
        #print('1')
        self.train_ds = self.Dataset(X_train, y_train,trans,device)
        print()
        #print('2')
        self.val_ds = self.Dataset(X_test, y_test,trans,device)
        #print('3')

    def train_dataloader(self):
        return DataLoader(self.train_ds, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val_ds, batch_size=self.batch_size)

In [ ]:
dm = BasuraDataModule()
dm.setup()

['/content/drive/MyDrive/IA2/Dataset_basura/metales/video5165726586179158716_066.jpg', '/content/drive/MyDrive/IA2/Dataset_basura/metales/video5165726586179158716_092.jpg', '/content/drive/MyDrive/IA2/Dataset_basura/metales/video5165726586179158716_102.jpg', '/content/drive/MyDrive/IA2/Dataset_basura/metales/video5165726586179158716_059.jpg', '/content/drive/MyDrive/IA2/Dataset_basura/metales/video5165726586179158716_057.jpg', '/content/drive/MyDrive/IA2/Dataset_basura/metales/video5165726586179158716_022.jpg', '/content/drive/MyDrive/IA2/Dataset_basura/metales/video5165726586179158716_045.jpg', '/content/drive/MyDrive/IA2/Dataset_basura/metales/video5165726586179158716_099.jpg', '/content/drive/MyDrive/IA2/Dataset_basura/metales/video5165726586179158716_027.jpg', '/content/drive/MyDrive/IA2/Dataset_basura/metales/video5165726586179158716_037.jpg', '/content/drive/MyDrive/IA2/Dataset_basura/metales/video5165726586179158716_053.jpg', '/content/drive/MyDrive/IA2/Dataset_basura/metales/vi

In [ ]:
imgs, labels = next(iter(dm.train_dataloader()))
#print(imgs.len())
imgs.shape, labels.shape

(torch.Size([64, 3, 28, 28]), torch.Size([64]))

In [ ]:
import torch.nn as nn

# https://github.com/jankrepl/mildlyoverfitted/blob/master/github_adventures/vision_transformer/custom.py

class PatchEmbedding(nn.Module):
    def __init__(self, img_size, patch_size, in_chans, embed_dim):
        super().__init__()
        self.img_size = img_size
        self.patch_size = patch_size
        self.n_patches = (img_size // patch_size) ** 2
        self.patch_size = patch_size
        self.proj = nn.Conv2d(in_chans, embed_dim, kernel_size=patch_size, stride=patch_size)#in_chans numeros de canales embed_dimen

    def forward(self, x):
        x = self.proj(x)  # (B, E, P, P)
        x = x.flatten(2)  # (B, E, N)
        x = x.transpose(1, 2)  # (B, N, E)
        return x

In [ ]:
pe = PatchEmbedding(28, 7, 3, 100)
out = pe(imgs)
out.shape

torch.Size([64, 16, 100])

In [ ]:
import math

class MultiHeadAttention(nn.Module):

    def __init__(self, n_embd, n_heads):
        super().__init__()
        self.n_heads = n_heads 
        
        # key, query, value projections
        self.key = nn.Linear(n_embd, n_embd*n_heads)
        self.query = nn.Linear(n_embd, n_embd*n_heads)
        self.value = nn.Linear(n_embd, n_embd*n_heads)
        
        # output projection
        self.proj = nn.Linear(n_embd*n_heads, n_embd)

    def forward(self, x):
        B, L, F = x.size()
        
        # calculate query, key, values for all heads in batch and move head forward to be the batch dim
        k = self.key(x).view(B, L, F, self.n_heads).transpose(1, 3) # (B, nh, L, F)
        q = self.query(x).view(B, L, F, self.n_heads).transpose(1, 3) # (B, nh, L, F)
        v = self.value(x).view(B, L, F, self.n_heads).transpose(1, 3) # (B, nh, L, F)
        
        # attention (B, nh, L, F) x (B, nh, F, L) -> (B, nh, L, L) 
        att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
        att = torch.nn.functional.softmax(att, dim=-1)
        y = att @ v # (B, nh, L, L) x (B, nh, L, F) -> (B, nh, L, F)
        y = y.transpose(1, 2).contiguous().view(B, L, F*self.n_heads) # re-assemble all head outputs side by side
        
        return self.proj(y)

In [ ]:
class TransformerBlock(nn.Module):
    def __init__(self, n_embd, n_heads):
        super().__init__()
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
        self.attn = MultiHeadAttention(n_embd, n_heads)
        self.mlp = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
        )
        
    def forward(self, x):
        x = x + self.attn(self.ln1(x))
        x = x + self.mlp(self.ln2(x))
        return x

In [ ]:
class ViT(nn.Module):

    def __init__(self, img_size=28, patch_size=7, in_chans=3, embed_dim=100, n_heads=3, n_layers=3, n_classes=6):
        super().__init__()

        self.patch_embed = PatchEmbedding(img_size, patch_size, in_chans, embed_dim)
        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        self.pos_embed = nn.Parameter(torch.zeros(1, 1 + self.patch_embed.n_patches, embed_dim))
        
        self.tranformer = torch.nn.Sequential(*[TransformerBlock(embed_dim, n_heads) for _ in range(n_layers)])
        
        self.ln = nn.LayerNorm(embed_dim)
        self.fc = torch.nn.Linear(embed_dim, n_classes)

    def forward(self, x):
        e = self.patch_embed(x)
        B, L, E = e.size()
        
        cls_token = self.cls_token.expand(B, -1, -1)  # (B, 1, E)
        e = torch.cat((cls_token, e), dim=1)  # (B, 1 + N, E)
        e = e + self.pos_embed  # (B, 1 + N, E)
        
        z = self.tranformer(e)
        
        cls_token_final = z[:, 0]  
        y = self.fc(cls_token_final)

        return y

In [ ]:
vit = ViT()
out = vit(imgs)
out.shape

torch.Size([64, 6])

In [ ]:
class Model(pl.LightningModule):

    def __init__(self):
        super().__init__()
        self.vit = ViT()

    def forward(self, x):
        return self.vit(x)

    def predict(self, x):
        with torch.no_grad():
          y_hat = self(x)
          return torch.argmax(y_hat, axis=1)
        
    def compute_loss_and_acc(self, batch):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        acc = (torch.argmax(y_hat, axis=1) == y).sum().item() / y.shape[0]
        return loss, acc
    
    def training_step(self, batch, batch_idx):
        loss, acc = self.compute_loss_and_acc(batch)
        self.log('loss', loss)
        self.log('acc', acc, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        loss, acc = self.compute_loss_and_acc(batch)
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_acc', acc, prog_bar=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.0003)
        return optimizer

In [ ]:
model = Model()
out = model(imgs)
out.shape

torch.Size([64, 6])

In [ ]:
model = Model()
trainer = pl.Trainer(max_epochs=5, gpus=1, logger=None)
trainer.fit(model, dm)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:430: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type | Params
------------------------------
0 | vit  | ViT  | 623 K 
------------------------------
623 K     Trainable params
0         Non-trainable params
623 K     Total params
2.492     Total estimated model params size (MB)


tensor([[[[0.8784, 0.8784, 0.8667,  ..., 0.8980, 0.9020, 0.9098],
          [0.9020, 0.9059, 0.9020,  ..., 0.2392, 0.6275, 0.1647],
          [0.1961, 0.6353, 0.1608,  ..., 0.4471, 0.3765, 0.2275],
          ...,
          [0.8431, 0.8314, 0.7255,  ..., 0.8157, 0.8078, 0.8157],
          [0.8118, 0.8000, 0.8078,  ..., 0.5725, 0.5529, 0.4078],
          [0.8118, 0.8196, 0.8157,  ..., 0.6824, 0.6784, 0.7608]],

         [[0.7608, 0.7608, 0.8000,  ..., 0.7922, 0.7922, 0.8000],
          [0.7961, 0.7961, 0.8039,  ..., 0.6706, 0.6118, 0.4510],
          [0.8196, 0.8196, 0.8196,  ..., 0.8196, 0.8196, 0.7961],
          ...,
          [0.7098, 0.7098, 0.7176,  ..., 0.4863, 0.3059, 0.2275],
          [0.7333, 0.7333, 0.7333,  ..., 0.7137, 0.7137, 0.7098],
          [0.7098, 0.7098, 0.7098,  ..., 0.6745, 0.6863, 0.6941]],

         [[0.6902, 0.6980, 0.7059,  ..., 0.3569, 0.2196, 0.1333],
          [0.7333, 0.7294, 0.7137,  ..., 0.7176, 0.7176, 0.7098],
          [0.7098, 0.7098, 0.7098,  ..., 0

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/data.py:133: UserWarning: Total length of `DataLoader` across ranks is zero. Please make sure this was your intention.
  f"Total length of `{dataloader.__class__.__name__}` across ranks is zero."


Training: 0it [00:00, ?it/s]

OSError: ignored